In [1]:
import pandas as pd
import numpy as np
from global_methods import *
import json

#### Correct JSON file

In [ ]:
with open("[PATH].json", 'rb') as f:
    tiled_json_genag_file = json.load(f)

with open("[PATH].json", 'rb') as f:
    tiled_json_removed_file_ = json.load(f)

for key_o in list(tiled_json_genag_file):
    if 'layers' not in key_o:
        if 'locked' in key_o.lower() or 'version' in key_o.lower() or 'visible' in key_o.lower():
            tiled_json_removed_file_[key_o] = tiled_json_genag_file[key_o]

for key_o in list(tiled_json_removed_file_):
    if 'layers' not in key_o:
        if key_o not in list(tiled_json_genag_file):
            if 'locked' in key_o.lower() or 'version' in key_o.lower() or 'visible' in key_o.lower():
                tiled_json_removed_file_.pop(key_o)

for i_r in range(len(tiled_json_removed_file_['layers'])):
    for i_r_key in list(tiled_json_removed_file_['layers'][i_r]):
        if i_r_key not in list(tiled_json_genag_file['layers'][i_r]):
            if 'locked' in i_r_key.lower() or 'version' in i_r_key.lower() or 'visible' in i_r_key.lower():
                tiled_json_removed_file_['layers'][i_r].pop(i_r_key)
        else:
            if tiled_json_removed_file_['layers'][i_r][i_r_key] != tiled_json_genag_file['layers'][i_r][i_r_key] and i_r_key != 'data':
                if 'locked' in i_r_key.lower() or 'version' in i_r_key.lower() or 'visible' in i_r_key.lower():
                    tiled_json_removed_file_['layers'][i_r] = tiled_json_genag_file['layers'][i_r]

with open('[PATH].json', 'w') as file:
    json.dump(tiled_json_removed_file_, file)

#### Make mapping dict from original tiled files

In [133]:
from_genag_coll = read_file_to_list('../matrix/maze_original/collision_maze.csv', header=False)[0]
from_genag_arenab = read_file_to_list('../matrix/maze_original/arena_maze.csv', header=False)[0]
from_genag_game_object = read_file_to_list('../matrix/maze_original/game_object_maze.csv', header=False)[0]
from_genag_sector = read_file_to_list('../matrix/maze_original/sector_maze.csv', header=False)[0]
from_genag_spawning_loc = read_file_to_list('../matrix/maze_original/spawning_location_maze.csv', header=False)[0]

with open("./the_ville.json", 'rb') as f:
    tiled_json_ori_file = json.load(f)

mapping_dict = {}

for key_, from_genag_ in [('collision', from_genag_coll), ('arena block', from_genag_arenab), ('object interaction block', from_genag_game_object), ('sector block', from_genag_sector), ('spawning block', from_genag_spawning_loc)]:
    mapping_dict[key_] = {}
    for i_ in range(len(tiled_json_ori_file['layers'])):
        if key_ not in tiled_json_ori_file['layers'][i_]['name'].lower():
            continue
        else:
            i_data = list(map(str, tiled_json_ori_file['layers'][i_]['data']))
            for k_ in range(len(i_data)):
                mapping_dict[key_][i_data[k_]] = from_genag_[k_]
            break
    
    for k_ in range(len(i_data)):
        if mapping_dict[key_][i_data[k_]] != from_genag_[k_]:
            print('here!!!!')
            break

In [134]:
import csv

with open("./the_ville_jan7-2.json", 'rb') as f:
    tiled_json_removed_file = json.load(f)

for key_ in ['collision', 'arena block', 'object interaction block', 'sector block', 'spawning block']:
    list_data = []
    for i_ in range(len(tiled_json_removed_file['layers'])):
        if key_ not in tiled_json_removed_file['layers'][i_]['name'].lower():
            continue
        else:
            i_data = list(map(str, tiled_json_removed_file['layers'][i_]['data']))
            for k_ in range(len(i_data)):
                list_data.append(mapping_dict[key_][i_data[k_]])
            break
    
    file_name_ = ''
    
    if key_ == 'collision':
        file_name_ = 'collision_maze.csv'
    elif key_ == 'arena block':
        file_name_ = 'arena_maze.csv'
    elif key_ == 'object interaction block':
        file_name_ = 'game_object_maze.csv'
    elif key_ == 'sector block':
        file_name_ = 'sector_maze.csv'
    elif key_ == 'spawning block':
        file_name_ = 'spawning_location_maze.csv'
    
    with open('../matrix/maze/' + file_name_, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(list_data)